In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from tqdm import tqdm

evd = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/evidence.json',orient='index')
print("evidence imported")
train = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/train-claims.json')
print("train imported") 
dev = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/dev-claims.json')
print("dev imported")
test = pd.read_json('/content/drive/MyDrive/Colab Notebooks/NLP project/FactChecker_NLP/data/test-claims-unlabelled.json')
print("test imported")

evd_list = evd.iloc[:, 0].tolist()
#load word2vec model
sentences = evd_list  

# # set batch size
# batch_size = 1000
# # lower case and split sentences into words
# preprocessed_sentences = [sentence.lower().split() for sentence in sentences]
# num_batches = (len(preprocessed_sentences) + batch_size - 1) // batch_size

# # initialize sentence embeddings
# sentence_embeddings = []



evidence imported
train imported
dev imported
test imported


Word2Vec embedding

In [ ]:
#load word2vec model
word2vec_model = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin.gz', binary=True)

# set batch size
batch_size = 1000
# lower case and split sentences into words
preprocessed_sentences = [sentence.lower().split() for sentence in sentences]
num_batches = (len(preprocessed_sentences) + batch_size - 1) // batch_size

# initialize sentence embeddings
sentence_embeddings = []

# iterate over batches
for i in tqdm(range(num_batches)):
    print('Processing batch {}'.format(i))
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(preprocessed_sentences))
    batch_sentences = preprocessed_sentences[batch_start:batch_end]

    for sentence in batch_sentences:
        embeddings = [word2vec_model[word] for word in sentence if word in word2vec_model]
        sentence_embedding = np.mean(embeddings, axis=0) if embeddings else np.zeros(300)
        sentence_embeddings.append(sentence_embedding)


In [2]:
!pip install Sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.5 MB/s eta 0:00:00


ALBERT embedding

In [8]:
import torch
from transformers import AlbertTokenizer, AlbertModel
import math
from sklearn.metrics.pairwise import cosine_similarity

# Check for CUDA support and set device
if torch.cuda.is_available():
    device = torch.device("cuda")
    # Release GPU cache
    torch.cuda.empty_cache()
    print(f"Using GPU: {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
    print("Using CPU")

# Load ALBERT model and tokenizer
model_name = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_name)
model = AlbertModel.from_pretrained(model_name).to(device)

# Function to get sentence embeddings using ALBERT
def albert_sentence_embedding(sentences, model, tokenizer, batch_size=128):
    all_embeddings = []
    num_batches = int(np.ceil(len(sentences) / batch_size))
    
    for batch_idx in range(num_batches):
        print('Processing batch {}'.format(batch_idx))
        print('left: {}'.format(num_batches - batch_idx))
        batch_start = batch_idx * batch_size
        batch_end = (batch_idx + 1) * batch_size
        batch_sentences = sentences[batch_start:batch_end]
        
        inputs = tokenizer(batch_sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)
        inputs = {key: val.to(model.device) for key, val in inputs.items()}
        
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()
        all_embeddings.extend(embeddings)
    
    return all_embeddings


# Initialize model and tokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-base-v2')
model.to('cuda')

# Calculate embeddings
sentence_embeddings = albert_sentence_embedding(sentences, model, tokenizer)


Using GPU: NVIDIA A100-SXM4-40GB


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight',

Processing batch 0
left: 9444
Processing batch 1
left: 9443
Processing batch 2
left: 9442
Processing batch 3
left: 9441
Processing batch 4
left: 9440
Processing batch 5
left: 9439
Processing batch 6
left: 9438
Processing batch 7
left: 9437
Processing batch 8
left: 9436
Processing batch 9
left: 9435
Processing batch 10
left: 9434
Processing batch 11
left: 9433
Processing batch 12
left: 9432
Processing batch 13
left: 9431
Processing batch 14
left: 9430
Processing batch 15
left: 9429
Processing batch 16
left: 9428
Processing batch 17
left: 9427
Processing batch 18
left: 9426
Processing batch 19
left: 9425
Processing batch 20
left: 9424
Processing batch 21
left: 9423
Processing batch 22
left: 9422
Processing batch 23
left: 9421
Processing batch 24
left: 9420
Processing batch 25
left: 9419
Processing batch 26
left: 9418
Processing batch 27
left: 9417
Processing batch 28
left: 9416
Processing batch 29
left: 9415
Processing batch 30
left: 9414
Processing batch 31
left: 9413
Processing batch 3

OutOfMemoryError: ignored

English sentences detector

In [ ]:
from langdetect import detect

#the length of all the sentences
print(len(sentences))

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

#get all the english sentences and non-english sentences and print the content
english_sentences = [sentence for sentence in sentences[0:10000] if is_english(sentence)]
non_english_sentences = [sentence for sentence in sentences[0:10000] if not is_english(sentence)]

print('English sentences:')
print(english_sentences[:10])
print('Non-English sentences:')
print(non_english_sentences[:10])


Similarity between sentences checker


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

#load the albert sentence embeddings
sentence_embeddings = np.load('../models/albert_embeddings.npy')

sentence_index = 990422
query_sentence_embedding = sentence_embeddings[sentence_index]

similarity_scores = cosine_similarity([query_sentence_embedding], sentence_embeddings)[0]

# 获取最相似的前10个句子的索引
top10_indices = np.argsort(similarity_scores)[-200:-1][::-1]  # 减去1以排除查询句子本身
#最不相似的前10个句子的索引
top10_indices2 = np.argsort(similarity_scores)[:10][::-1]  # 减去1以排除查询句子本身

print("average similarity score of sentence"+str(sentence_index)+":")
print(np.mean(similarity_scores))

print("median similarity score of sentence"+str(sentence_index)+":")
print(np.median(similarity_scores))

print("Top 10 closest sentences to sentence"+str(sentence_index)+":")
rank = 1
for index in top10_indices:
    print("rank"+str(rank))
    rank += 1
    print(f"Index: {index}, Similarity Score: {similarity_scores[index]}")
    print("===========================================")
    print(f"original sentence: {evd_list[sentence_index]}")
    print(f"Sentence: {sentences[index]}\n")


# print("Top 10 furthest sentences to sentence"+str(sentence_index)+":")
# for index in top10_indices2:
#     print(f"Index: {index}, Similarity Score: {similarity_scores[index]}")
#     print("===========================================")
#     print(f"original sentence: {evd_list[sentence_index]}")
#     print(f"Sentence: {sentences[index]}\n")


#print the distance to of sentence index to sentence 2 index
sentence2_index = 920671
print("distance between sentence"+str(sentence_index)+" and sentence"+str(sentence2_index)+":")
print(cosine_similarity([sentence_embeddings[sentence_index]], [sentence_embeddings[sentence2_index]]))






average similarity score of sentence990422:
0.75711393
median similarity score of sentence990422:
0.76449865
Top 10 closest sentences to sentence990422:
rank1
Index: 103064, Similarity Score: 0.9441030621528625
original sentence: In May 2014, a series of at least 20 wildfires broke out in San Diego County during severe Santa Ana Wind conditions, historic drought conditions, and a heat wave.
Sentence: The storm arrived just two months after Hurricane Irene caused extensive power outages and property damage in the Northeast ; with the 2011 New England tornado outbreak also causing damage in Western Massachusetts.

rank2
Index: 593586, Similarity Score: 0.9435477256774902
original sentence: In May 2014, a series of at least 20 wildfires broke out in San Diego County during severe Santa Ana Wind conditions, historic drought conditions, and a heat wave.
Sentence: On the morning of June 27, 2015, heavy rain occurred in a great portion of the Scouting camp Philmont Scout Ranch, which is near 